---
- title: JS Promises
- author: david
- categories: [Lab Notebook]
- tags: [Project, HTML, Javascript]
- type: tangibles
- week: 10
- description: async and promise tests
- toc: True
- comments: True
- date: 2023-10-27 12:00:00 +0000
---

In [ ]:
// get path from backend
function getElementPath() {
  // debug
  console.log('Current start and end values:', start, end);
  console.log('Current adj_List:', adj_List);

  // create adjacency list by filling all with infinity
  adj_List = Array(current_index).fill().map(() => Array(current_index).fill(10000));
  // using weight data from second adj list for main adj list
  for (let i = 0; i < adj_array.length; i++) {
    adj_List[adj_array[i][0]][adj_array[i][1]] = adj_array[i][2];
    adj_List[adj_array[i][1]][adj_array[i][0]] = adj_array[i][2];
  }
  // setting empty values with 0 for nodes connected to themselves
  for (let i = 0; i < current_index; i ++){
    adj_List[i][i] = 0;
  }

  // defines data set to backend
  const payload = {
      adjacencyList: adj_List,
      source: parseInt(start),
      target: parseInt(end)
  };

  // Backend URL
  const backendURL = 'http://localhost:8084/api/dijkstra/';

  // Creating a new XMLHttpRequest object
  const xhr = new XMLHttpRequest();
  xhr.open('POST', backendURL, true);
  xhr.setRequestHeader('Content-Type', 'application/json');

  // Handling the response from the server
  // uses promises to wait for response before displaying
  return new Promise((resolve) => {
      xhr.onload = function() {
      
          if(xhr.status >= 200 && xhr.status < 300) {
              const response = JSON.parse(xhr.responseText);
              console.log('Response from server:', response);
              
              resolve(response)
          } else {
              console.error('Request failed with status:', xhr.status);
          }
          };
      
          // Handling errors during the request
          xhr.onerror = function() {
          console.error('Request failed');
          };
      
          // Sending the request with the JSON payload
          xhr.send(JSON.stringify(payload));
  });
}

In [ ]:
// showing path
async function showPath() {

  // sets element path from data from backend, awaiting for response
  const elementPath = await getElementPath();
  // checks if path is found
  const isPathFound = elementPath.length > 0;
  // highlights for non-existent path
  if (!isPathFound && startView && endView && startView.id !== endView.id && !editMode) {
      joint.highlighters.addClass.add(startView, 'body', invalidPathHighlightId, {
          className: invalidPathClassName
      });
      joint.highlighters.addClass.add(endView, 'body', invalidPathHighlightId, {
          className: invalidPathClassName
      });
      // doesn't show path
      hidePath();
      return;
  }

  // removes if start and end don't exist
  if (startView) joint.highlighters.addClass.remove(startView, invalidPathHighlightId);
  if (endView) joint.highlighters.addClass.remove(endView, invalidPathHighlightId);
  hidePath();

  // gets link path between node path
  const linkPath = getLinkPath(elementPath);
  // display all elements
  for (const elementId of [...elementPath, ...linkPath]) {
      const element = graph.getCell(elementId);
      const view = element.findView(paper);
      const isLink = view.model.isLink();
      // styles nodes
      joint.highlighters.addClass.add(view, isLink ? 'line' : 'body', pathMemberHighlightId, {
          className: pathMemberClassName
      });
      // sets link styles
      if (isLink) {
          element.set('z', 2);
      }

      // creates path members array
      pathMembersViews.push(view);
  }
}